### Function 'flights' created to build a pipeline extracting from mySQL remote host into this notebook and give back a dataframe of scheduled arrivals.

In [1]:
import pandas as pd
import requests

### 1. Connection made with local host in mySQL

In [ ]:
#from keys import mySQL_pass
from keys import API_key_aerodata, mySQL_pass # personal passwords!

schema="gans" # name of the database you want to use here
host="localhost" # to connect to your local server
user="root"
password=mySQL_pass # your password!!!!
port=3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

### 2. Connection made with remote host in mySQL

In [5]:
from con_aws_to_sql import aws_pass

schema="gans"  # name of the database you want to use here
host='wbs-project4-db.c1iajchyyr27.eu-north-1.rds.amazonaws.com'
username = 'admin'
port = 3306
password = aws_pass # Amazon RDS in mySQL
con = f'mysql+pymysql://{username}:{password}@{host}:{port}/{schema}'

### 3. Extracting the info of airports icaos for the function to trigger

In [10]:
airport_icaos = pd.read_sql("cities_airports", con=con)
list_icao = airport_icaos['airport_icao'].tolist()

### 4. The function 'flights'

In [11]:
def flights(list_icao):
    #this function takes a list of 'icao' and return a dataframe of all flights arriving at that airport on predefined time.
    
    #import pandas as pd
    #import requests
    from datetime import datetime, date, timedelta
    from pytz import timezone, utc
    from keys import API_key_aerodata

    #airport_icaos = pd.read_sql("cities_airports", con=con)
    #list_icao = airport_icaos['airport_icao'].tolist()

    today = datetime.now().astimezone(timezone('Europe/Berlin')).date()
    tomorrow = (today + timedelta(days=1))
    
    arr_airport_icao = []
    dep_airport_icao = []
    dep_airport_name = []
    arr_flight_schtime = []
    arr_flight_revtime = []
    arr_flight_number = []
    arr_flight_status = []
    arr_flight_model = []
    arr_flight_name = []
    
    for icao in list_icao:
        url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao}/{tomorrow}T08:00/{tomorrow}T19:59"

        querystring = {"withLeg":"false","direction":"Arrival","withCancelled":"false","withCargo":"true","withPrivate":"true","withLocation":"false"}

        headers = {
            "X-RapidAPI-Key": API_key_aerodata,
            "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
        }

        response = requests.get(url, headers=headers, params=querystring)
        flight_info = response.json()

        if 'arrivals' in flight_info and flight_info['arrivals']:
            for i in range(len(flight_info['arrivals'])):
                try:
                    revised_time_local = flight_info['arrivals'][i]['movement']['revisedTime']['local']
                    revised_time = datetime.strptime(revised_time_local, '%Y-%m-%d %H:%M%z')
                    revised_time_utc = revised_time.astimezone(utc).replace(tzinfo=None)
                    arr_flight_revtime.append(revised_time_utc)
                except:
                    arr_flight_revtime.append(None)
                try: 
                    aircraft = flight_info['arrivals'][i]['aircraft']['model']
                    arr_flight_model.append(aircraft)
                except:
                    arr_flight_model.append(None)
                try:
                    arrival_icao = flight_info['arrivals'][i]['movement']['airport']['icao']
                    dep_airport_icao.append(arrival_icao)
                except:
                    dep_airport_icao.append(None)
                arr_airport_icao.append(icao)
                dep_airport_name.append(flight_info['arrivals'][i]['movement']['airport']['name'])
                scheduled_time_local = flight_info['arrivals'][i]['movement']['scheduledTime']['local']
                scheduled_time = datetime.strptime(scheduled_time_local, '%Y-%m-%d %H:%M%z')
                scheduled_time_utc = scheduled_time.astimezone(utc).replace(tzinfo=None)
                arr_flight_schtime.append(scheduled_time_utc)
                arr_flight_number.append(flight_info['arrivals'][i]['number'])
                arr_flight_status.append(flight_info['arrivals'][i]['status'])
                #arr_flight_model.append(flight_info['arrivals'][i]['aircraft']['model'])
                arr_flight_name.append(flight_info['arrivals'][i]['airline']['name'])
        else:
            print(f'No arrivals found for ICAO: {icao}')
    arr_columns = [
            'airport_icao'
            ,'departure_airport_icao'
            ,'departure_airport_name'
            ,'scheduled_time'
            ,'revised_time'
            ,'flight_number'
            ,'flight_status'
            ,'aircraft_used'
            ,'airline_name'
        ]

    return pd.DataFrame(list(zip( arr_airport_icao
        ,dep_airport_icao                        
        ,dep_airport_name
        ,arr_flight_schtime
        ,arr_flight_revtime
        ,arr_flight_number
        ,arr_flight_status
        ,arr_flight_model
        ,arr_flight_name)),columns=arr_columns)


In [12]:
flights(list_icao)

,airport_icao,departure_airport_icao,departure_airport_name,scheduled_time,revised_time,flight_number,flight_status,aircraft_used,airline_name
0,EDDB,EFHK,Helsinki,2023-12-07 07:00:00,2023-12-07 07:00:00,AY 1431,Expected,Airbus A321-200 (sharklets),Finnair
1,EDDB,EDDS,Stuttgart,2023-12-07 07:00:00,2023-12-07 07:00:00,EW 8001,Expected,Airbus A321-100,Eurowings
2,EDDB,EDDS,Stuttgart,2023-12-07 07:00:00,2023-12-07 07:00:00,LH 5392,Expected,Airbus A321-100,Lufthansa
3,EDDB,EDDR,Saarbrücken,2023-12-07 07:05:00,2023-12-07 07:05:00,DX 121,Expected,ATR 42-300,Danish Air
4,EDDB,LIRF,Rome,2023-12-07 07:10:00,2023-12-07 07:10:00,FR 41,Expected,Boeing 737-800 (winglets),Ryanair
...,...,...,...,...,...,...,...,...,...
4149,EKCH,ELLX,Luxembourg,2023-12-07 18:50:00,2023-12-07 18:50:00,LG 5435,Expected,Bombardier Dash 8 Q400 / DHC-8-400,Luxair
4150,EKCH,LSGG,Geneva,2023-12-07 18:50:00,2023-12-07 18:50:00,U2 1539,Expected,Airbus A320,easyJet
4151,EKCH,ESSA,Stockholm,2023-12-07 18:55:00,2023-12-07 18:55:00,D8 3213,Expected,Boeing 737,Norwegian Air Sweden
4152,EKCH,EGLL,London,2023-12-07 18:55:00,2023-12-07 18:55:00,SK 506,Expected,Airbus A320 NEO,SAS
